In [1]:
from graphs import *
from arch import arch_model
from scipy.stats import mstats
from datetime import datetime as dt
import numpy as np
import plotly.graph_objs as go
import plotly.subplots as sp
from datetime import date as dt 
import pandas as pd

file_path= r'C:\Users\Clement\OneDrive\Documents\dashboard_data.xlsx'

In [2]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 600,
    'width': 800,
    'scale': 7  }} # Multiply title/legend/axis/canvas sizes by this factor

In [40]:
export=pd.read_excel(file_path,sheet_name="export_china")
fig=go.Figure()
fig.add_trace(go.Scatter(x=export['Date'],y=export['VALUE'],line_color='#00334E',name='Exports'))
    # add horizontal line
fig.update_layout(title={'text': "Exportations de la Chine (YoY)",'x': 0.5,'xanchor': 'center'},xaxis_title='Date',template="simple_white",
    legend=dict(x=0, y=-0.2, orientation='h', bgcolor='rgba(255, 255, 255, 0)'))
fig

In [41]:
def cpi_china(start_date='1990-01-01',end_date=dt.today()):
    """
    Function
    ---------- 
    Plot with plotly the consumer price index rate in the us
    https://fred.stlouisfed.org/series/CPALTT01CNM659N
    
    Parameters
    ----------
    start_date: start date of you're plot
    end_date: end date of you're plot
    
    Returns
    ------
    plot the CPI in US
    """
    cpi=web.DataReader('CPALTT01CNM659N','fred', start=start_date, end=end_date)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=cpi.index, y=cpi['CPALTT01CNM659N'],line_color='#00334E', name='IPC'))
    fig.update_layout(title={'text': 'Indice des prix à la consommation','x': 0.5,'xanchor': 'center'},xaxis_title='Date',yaxis_title='IPC (%)',template="simple_white",
                      legend=dict(x=0, y=-0.2, orientation='h', bgcolor='rgba(255, 255, 255, 0)'))
    return fig